In [1]:
from model import DQN
import os
import minerl
import gym
import torch
import torch.nn.functional as F
import torch.optim as optim
import wandb
import ray

/home/kukjin/anaconda3/envs/minerl/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)
/home/kukjin/anaconda3/envs/minerl/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
from _collections import deque
from utils import *
import random

In [3]:
from subprocess import call
wandb.init(project='apex_dqfd', entity='neverparadise')

wandb: Currently logged in as: neverparadise (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.30 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
#하이퍼 파라미터
learning_rate = 0.0003
gamma = 0.999
buffer_limit = 50000
L1 = 0.9
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
def margin_loss(q_value, action, demo, weigths):
    ae = F.one_hot(action, num_classes=19)
    zero_indices = (ae == 0)
    one_indices = (ae == 1)
    ae[zero_indices] = 1
    ae[one_indices] = 0
    ae = ae.to(float)
    max_value = torch.max(q_value + ae, axis=1)

    ae = F.one_hot(action, num_classes=19)
    ae = ae.to(float)

    J_e = torch.abs(torch.sum(q_value * ae,axis=1) - max_value.values)
    J_e = torch.mean(J_e * weigths * demo)
    return J_e

In [7]:
from st import SumTree
@ray.remote
class Memory:  # stored as ( s, a, r, s_, n_rewards ) in SumTree
    e = 0.01
    a = 0.6
    beta = 0.4
    beta_increment_per_sampling = 0.001

    def __init__(self, capacity):
        self.tree = SumTree(capacity)
        self.capacity = capacity

    def _get_priority(self, error):
        return (np.abs(error) + self.e) ** self.a

    def add(self, error, sample):
        p = self._get_priority(error)
        self.tree.add(p, sample)

    def size(self):
        return self.tree.n_entries

    def sample(self, n):
        batch = []
        idxs = []
        segment = self.tree.total() / n
        priorities = []

        self.beta = np.min([1., self.beta + self.beta_increment_per_sampling])

        for i in range(n):
            a = segment * i
            b = segment * (i + 1)

            s = random.uniform(a, b)
            (idx, p, data) = self.tree.get(s)
            priorities.append(p)
            batch.append(data)
            idxs.append(idx)

        sampling_probabilities = priorities / self.tree.total()
        is_weight = np.power(self.tree.n_entries * sampling_probabilities, -self.beta)
        is_weight /= is_weight.max() + 1e-5

        return batch, idxs, is_weight

    def update(self, idx, error):
        p = self._get_priority(error)
        self.tree.update(idx, p)

In [5]:
def train_dqn(policy_net, target_net, demos, batch_size, demo_prob, optimizer):
    demo_batch, idxs, is_weights = demos.sample.remote(batch_size)
    # demo_batch = (batch_size, state, action, reward, next_state, done, n_rewards)
    #print(len(demo_batch[0])) # 0번째 배치이므로 0이 나옴
    state_list = []
    action_list = []
    reward_list =[]
    next_state_list = []
    done_mask_list = []
    n_rewards_list = []

    for transition in demo_batch:
        s, a, r, s_prime, done_mask, n_rewards = transition
        state_list.append(s)
        action_list.append([a])
        reward_list.append([r])
        next_state_list.append(s_prime)
        done_mask_list.append([done_mask])
        n_rewards_list.append([n_rewards])

    #a = state_list
    #b = torch.tensor(action_list, dtype=torch.int64)
    #c = torch.tensor(reward_list)
    #d = next_state_list
    #e = torch.tensor(done_mask_list)
    #f = torch.tensor(n_rewards_list)

    s = torch.stack(state_list).float().to(device)
    a = torch.tensor(action_list, dtype=torch.int64).to(device)
    r =  torch.tensor(reward_list).to(device)
    s_prime = torch.stack(next_state_list).float().to(device)
    done_mask = torch.tensor(done_mask_list).float().to(device)
    nr =  torch.tensor(n_rewards_list).to(device)

    q_vals = policy_net(s)
    state_action_values = q_vals.gather(1, a)

    # comparing the q values to the values expected using the next states and reward
    next_state_values = target_net(s_prime).max(1)[0].unsqueeze(1)
    target = r + (next_state_values * gamma)

    # calculating the q loss, n-step return lossm supervised_loss
    is_weights = torch.FloatTensor(is_weights).to(device)
    q_loss = (is_weights * F.mse_loss(state_action_values, target)).mean()
    n_step_loss = (state_action_values.max(1)[0] + nr).mean()
    supervised_loss = margin_loss(q_vals, a, 1, 1)

    loss = q_loss + supervised_loss + n_step_loss
    wandb.log({"Q-loss" : q_loss.item()})
    wandb.log({"n-step loss" : n_step_loss.item()})
    wandb.log({"super_vised loss" : supervised_loss.item()})
    wandb.log({"total loss" : loss.item()})
    
    errors = torch.abs(state_action_values - target).data.cpu()
    errors = errors.numpy()
    # update priority
    for i in range(batch_size):
        idx = idxs[i]
        demos.update.remote(idx, errors[i])

    # optimization step and logging
    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm(policy_net.parameters(), 100)
    optimizer.step()
    return loss

In [7]:
def append_sample(memory, model, target_model, state, action, reward, next_state, done, n_rewards):
    # Caluclating Priority (TD Error)
    target = model(state.float()).data
    old_val = target[0][action].cpu()
    target_val = target_model(next_state.float()).data.cpu()
    if done:
        target[0][action] = reward
    else:
        target[0][action] = reward + 0.99 * torch.max(target_val)

    error = abs(old_val - target[0][action])
    error = error.cpu() 
    memory.add(error, [state, action, reward, next_state, done, n_rewards])


In [8]:
def pre_train(env_name, rep_buffer, policy_net, target_net, optimizer,threshold=10, num_epochs=1, batch_size=16, seq_len=10, gamma=0.99):
    data = minerl.data.make(env_name)
    print("data loading sucess")
    demo_num = 0
    for s_batch, a_batch, r_batch, ns_batch, d_batch in data.batch_iter(num_epochs=num_epochs, batch_size=batch_size,
                                                                        seq_len=seq_len):
        demo_num += 1
        print(demo_num)
        if r_batch.sum() < threshold:
            continue
        """
        state_batch : (batch_size, seq_len, 64, 64, 3)
        action_batch : (batch_size, seq_len, action['property'].shape) ex camera = 2 otherwise 1

        reward_batch : (batch_size, seq_len)
        next_state_batch : (batch_size, seq_len, 64, 64, 3)
        done_batch : (batch_size, seq_len)

    
        reward, _ = stats.mode(r_batch, axis=1)
        reward = np.squeeze(reward)
        done, _ = stats.mode(d_batch, axis=1)
        done = np.squeeze(done)
        """
        parse_ts = 0

        # 각 state에 대한 action discretize를 위해 반복문을 사용
        batch_length = (s_batch['pov'].shape)[0]  # (batch, seq, 64, 64, 3)[0]
        for i in range(0, batch_length):
            episode_start_ts = 0

            n_step = 10
            n_step_state_buffer = deque(maxlen=n_step)
            n_step_action_buffer = deque(maxlen=n_step)
            n_step_reward_buffer = deque(maxlen=n_step)
            n_step_n_rewards_buffer = deque(maxlen=n_step)
            n_step_next_state_buffer = deque(maxlen=n_step)
            n_step_done_buffer = deque(maxlen=n_step)
            gamma_list = [gamma ** i for i in range(n_step)]

            for j in range(0, seq_len):
                av = a_batch['attack'][i][j]  # attack value
                aj = a_batch['jump'][i][j]  # jump value
                af = a_batch['forward'][i][j]  # forward value
                ab = a_batch['back'][i][j]  # back value
                al = a_batch['left'][i][j]  # left value
                ar = a_batch['right'][i][j]  # right value
                va = a_batch['camera'][i][j][0]  # vertical angle and
                ha = a_batch['camera'][i][j][1]  # horizontal angle

                camera_thresholds = (abs(va) + abs(ha)) / 2.0
                # 카메라를 움직이는 경우
                if (camera_thresholds > 2.5):
                    # camera = [0, -5]
                    if abs(va) < abs(ha) and ha < 0:
                        if av == 0:
                            action_index = 0
                        else:
                            action_index = 1
                    # camera = [0, 5]
                    elif abs(va) < abs(ha) and ha > 0:
                        if av == 0:
                            action_index = 2
                        else:
                            action_index = 3
                    # camera = [-5, 0]
                    elif abs(va) > abs(ha) and ha < 0:
                        if av == 0:
                            action_index = 4
                        else:
                            action_index = 5
                    # camera = [5, 0]
                    elif abs(va) > abs(ha) and ha > 0:
                        if av == 0:
                            action_index = 6
                        else:
                            action_index = 7

                            # 카메라를 안움직이는 경우
                # 점프하는 경우
                elif (aj == 1):
                    if (af == 0):
                        action_index = 8
                    else:
                        action_index = 9

                # 앞으로 가는 경우
                elif (af == 1):
                    if (av == 0):
                        action_index = 10
                    else:
                        action_index = 11

                # 뒤로 가는 경우
                elif (ab == 1):
                    if (av == 0):
                        action_index = 12
                    else:
                        action_index = 13

                # 왼쪽으로 가는 경우
                elif (al == 1):
                    if (av == 0):
                        action_index = 14
                    else:
                        action_index = 15

                # 오른쪽으로 가는 경우
                elif (ar == 1):
                    if (av == 0):
                        action_index = 16
                    else:
                        action_index = 17

                # 카메라, 움직임이 다 0이고 공격만 하는 것
                else:
                    if (av == 0):
                        continue
                    else:
                        action_index = 18

                a_index = torch.LongTensor([action_index]).cpu()
                curr_obs = converter2(s_batch['pov'][i][j]).float().cpu()
                _obs = converter2(ns_batch['pov'][i][j]).float().cpu()
                _reward = torch.FloatTensor([r_batch[i][j]]).cpu()
                _done = d_batch[i][j]  # .astype(int)

                n_step_state_buffer.append(curr_obs)
                n_step_action_buffer.append(a_index)
                n_step_reward_buffer.append(_reward)
                n_step_next_state_buffer.append(_obs)
                n_step_done_buffer.append(_done)
                n_rewards = sum([gamma * reward for gamma, reward in zip(gamma_list, n_step_reward_buffer)])
                n_step_n_rewards_buffer.append(n_rewards)
                

                append_sample(rep_buffer, policy_net, target_net, n_step_state_buffer[j], \
                              n_step_action_buffer[j], n_step_reward_buffer[j], \
                              n_step_next_state_buffer[j], \
                              n_step_done_buffer[j], \
                              n_step_n_rewards_buffer[j])
                episode_start_ts += 1
                parse_ts += 1
                # if episode done we reset
                if _done:
                    break

        # replay is over emptying the deques
        #if rep_buffer.size() > rep_buffer.buffer_limit:
        #    rep_buffer.buffer.popleft()
        print('Parse finished. {} expert samples added.'.format(parse_ts))
        train_dqn(policy_net, target_net, rep_buffer, batch_size, 1, optimizer)
        torch.save(policy_net.state_dict(), model_path + 'pre_trained3.pth')        
        if demo_num % 5 == 0 and demo_num != 0:
        # 특정 반복 수가 되면 타겟 네트워크도 업데이트
            print("target network updated")
            target_net.load_state_dict(policy_net.state_dict())
        print("train {} step finished".format(demo_num))
    print('pre_train finished')
    return rep_buffer


In [9]:
total_episodes = 1000
startEpsilon = 1.0
endEpsilon = 0.05
epsilon = startEpsilon

root_path = os.curdir
model_path = root_path + '/dqn_model/'

stepDrop = (startEpsilon - endEpsilon) / total_episodes

In [10]:
@ray.remote(num_cpus=6)
class Actor:
    def __init__(self, shared_network_cpu, actor_idx, epsilon):
        # environment initialization
        import gym
        import minerl
        self.actor_idx = actor_idx
        self.env = gym.make("MineRLTreechop-v0")
        self.port_number = int("12340")+actor_idx
        print("actor environment %d initialize successfully" % self.actor_idx)
        
        #self.shared_memory = ray.get(shared_memory_id)
        #print("shared memory assign successfully")
        
        # network initalization
        self.actor_network = DQN(19).cpu()
        self.actor_target_network = DQN(19).cpu()
        self.actor_network.load_state_dict(shared_network_cpu.state_dict())
        self.actor_target_network.load_state_dict(self.actor_network.state_dict())
        print("actor network %d initialize successfully" % self.actor_idx)
        
        # exploring info
        self.epsilon = epsilon
        self.max_step = 10
        self.local_buffer_size = 100
        self.local_buffer = deque(maxlen=self.local_buffer_size)
        
    # 1. 네트워크 파라미터 복사
    # 2. 환경 탐험 (초기화, 행동)
    # 3. 로컬버퍼에 저장
    # 4. priority 계산
    # 5. 글로벌 버퍼에 저장
    # 6. 주기적으로 네트워크 업데이트 
    
    # 각 환경 인스턴스에서 각 엡실론에 따라 탐험을 진행한다.
    # 탐험 과정에서 local buffer에 transition들을 저장한다.
    # local buffer의 개수가 특정 개수 이상이면 global buffer에 추가해준다. 
    def make_inveractive(self):
        self.env.make_interactive(port=self.port_number, realtime=False)
        
    def explore(self, shared_network, shared_memory):
        self.env.make_interactive(port=self.port_number, realtime=False)
        for num_epi in range(self.max_step):
            obs = self.env.reset()
            state = converter(obs).cpu()
            state = state.float()
            done = False
            total_reward = 0
            steps = 0
            total_steps = 0
            self.epsilon = 0.5
            if(self.epsilon > endEpsilon):
                self.epsilon -= stepDrop / (self.actor_idx + 1)
            
            n_step = 10
            n_step_state_buffer = deque(maxlen=n_step)
            n_step_action_buffer = deque(maxlen=n_step)
            n_step_reward_buffer = deque(maxlen=n_step)
            n_step_n_rewards_buffer = deque(maxlen=n_step)
            n_step_next_state_buffer = deque(maxlen=n_step)
            n_step_done_buffer = deque(maxlen=n_step)
            gamma_list = [gamma ** i for i in range(n_step)]
            
            while not done:
                steps += 1
                total_steps += 1
                a_out = self.actor_network.sample_action(state,self.epsilon)
                action_index = a_out
                action = make_action(self.env, action_index)
                action['attack'] = 1
                obs_prime, reward, done, info = self.env.step(action)
                total_reward += reward
                state_prime = converter(obs_prime)
                
                # local buffer add
                n_step_state_buffer.append(state)
                n_step_action_buffer.append(action_index)
                n_step_reward_buffer.append(reward)
                n_step_next_state_buffer.append(state_prime)
                n_step_done_buffer.append(done)
                n_rewards = sum([gamma * reward for gamma, reward in zip(gamma_list, n_step_reward_buffer)])
                n_step_n_rewards_buffer.append(n_rewards)
                
                if (len(n_step_state_buffer) >= n_step):
                    # LocalBuffer Get
                    # Compute Priorities
                    for i in range(n_step):
                        self.append_sample(shared_memory, self.actor_network , self.actor_target_network, \
                              n_step_state_buffer[i], \
                              n_step_action_buffer[i], n_step_reward_buffer[i], \
                              n_step_next_state_buffer[i], \
                              n_step_done_buffer[i], \
                              n_step_n_rewards_buffer[i])
                        if(n_step_done_buffer[i]):
                            break
                print((shared_memory.size()))                
                
                    
                state = state_prime.float().cpu()
                if done:
                    break

            if done:
                print("%d episode is done" % num_epi)
                print("total rewards : %d " % total_reward)
            
            if (num_epi % 5 == 0 and num_epi != 0):
                self.update_params(shared_network)
                print("actor network is updated ")
                
    def env_close(self):
        self.env.close()

    def update_params(self, shared_network):
        self.actor_network.load_state_dict(shared_network.state_dict())
        
    def append_sample(self, memory, model, target_model, state, action, reward, next_state, done, n_rewards):
        # Caluclating Priority (TD Error)
        target = model(state.float()).data
        old_val = target[0][action].cpu()
        target_val = target_model(next_state.float()).data.cpu()
        if done:
            target[0][action] = reward
        else:
            target[0][action] = reward + 0.99 * torch.max(target_val)

        error = abs(old_val - target[0][action])
        error = error.cpu() 
        memory.add(error, [state, action, reward, next_state, done, n_rewards])



        

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [12]:
print(device)

cuda:0


In [13]:
print(torch.cuda.is_available())

True


In [14]:
class Learner:
    def __init__(self, network, batch_size):
        self.learner_network = DQN(19).to(device)
        self.learner_target_network = DQN(19).to(device)
        
        self.learner_network.load_state_dict(network.state_dict())
        self.learner_target_network.load_state_dict(network.state_dict())
        
        self.batch_size = batch_size
    # 1. sampling
    # 2. calculate gradient
    # 3. weight update
    # 4. compute priorities
    # 5. priorities of buffer update
    # 6. remove old memory 
    def get_network(self):
        return self.learner_network

    def update_network(memory, demos, batch_size, demo_prob, optimizer):
        
        agent_batch, agent_idxs, agent_weights = memory.sample(batch_size)
        demo_batch, demo_idxs, demo_weights = demos.sample(batch_size)
        
        # demo_batch = (batch_size, state, action, reward, next_state, done, n_rewards)
        #print(len(demo_batch[0])) # 0번째 배치이므로 0이 나옴
        state_list = []
        action_list = []
        reward_list =[]
        next_state_list = []
        done_mask_list = []
        n_rewards_list = []

        for agent_transition, expert_transition in zip(agent_batch, demo_batch):
            s, a, r, s_prime, done_mask, n_rewards = agent_transition
            state_list.append(s)
            action_list.append([a])
            reward_list.append([r])
            next_state_list.append(s_prime)
            done_mask_list.append([done_mask])
            n_rewards_list.append([n_rewards])
            
            s, a, r, s_prime, done_mask, n_rewards = expert_transition
            state_list.append(s)
            action_list.append([a])
            reward_list.append([r])
            next_state_list.append(s_prime)
            done_mask_list.append([done_mask])
            n_rewards_list.append([n_rewards])
            

        s = torch.stack(state_list).float().to(device)
        a = torch.tensor(action_list, dtype=torch.int64).to(device)
        r =  torch.tensor(reward_list).to(device)
        s_prime = torch.stack(next_state_list).float().to(device)
        done_mask = torch.tensor(done_mask_list).float().to(device)
        nr =  torch.tensor(n_rewards_list).to(device)
        
        q_vals = policy_net(s)
        state_action_values = q_vals.gather(1, a)

        # comparing the q values to the values expected using the next states and reward
        next_state_values = target_net(s_prime).max(1)[0].unsqueeze(1)
        target = r + (next_state_values * gamma)

        # calculating the q loss, n-step return lossm supervised_loss
        is_weights = torch.FloatTensor(is_weights).to(device)
        q_loss = (is_weights * F.mse_loss(state_action_values, target)).mean()
        n_step_loss = (state_action_values.max(1)[0] + nr).mean()
        supervised_loss = margin_loss(q_vals, a, 1, 1)

        loss = q_loss + supervised_loss + n_step_loss
        wandb.log({"Q-loss" : q_loss.item()})
        wandb.log({"n-step loss" : n_step_loss.item()})
        wandb.log({"super_vised loss" : supervised_loss.item()})
        wandb.log({"total loss" : loss.item()})

        errors = torch.abs(state_action_values - target).data.cpu()
        errors = errors.numpy()
        # update priority
        for i in range(batch_size):
            idx = idxs[i]
            memory.remote.update(idx, errors[i])

        # optimization step and logging
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm(policy_net.parameters(), 100)
        optimizer.step()
        return loss
    
    def update_target_networks(self):
        self.learner_target_network.load_state_dict(self.learner_network.state_dict())


In [15]:
ray.init()

2021-05-27 02:30:14,735	INFO services.py:1269 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.0.22',
 'raylet_ip_address': '192.168.0.22',
 'redis_address': '192.168.0.22:54860',
 'object_store_address': '/tmp/ray/session_2021-05-27_02-30-13_854772_36645/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-05-27_02-30-13_854772_36645/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-05-27_02-30-13_854772_36645',
 'metrics_export_port': 59064,
 'node_id': 'a1643c12189b9baaa2cf914ad72c90c3dc61dc2f886d70c2f30a7a94'}

In [16]:
policy_net = DQN(19).to(device=device)
target_net = DQN(19).to(device=device)
target_net.load_state_dict(policy_net.state_dict())
memory = Memory(50000)
demos = Memory(50000)
optimizer = optim.Adam(policy_net.parameters(), lr=learning_rate, weight_decay=1e-5)

In [17]:
#print("pre_train start")
#demos = pre_train("MineRLTreechop-v0", demos, policy_net, target_net, optimizer, threshold=60, num_epochs=10, batch_size=256, seq_len=10, gamma=0.99)
#print("pre_train finished")
#print(demos.size.remote())

In [18]:
# Copy network params from pretrained Agent
model_path = './dqn_model/pre_trained.pth'
policy_net.load_state_dict(torch.load(model_path, map_location='cuda:0'))

<All keys matched successfully>

In [19]:
def get_network(shared_network_id):
    shared_network = ray.get(shared_network_id).get_network()
    return shared_network.cpu()


In [20]:
# Generating each own instances
# main()
num_actors = 2
epsilon = 0.5

# learner network initialzation
learner = Learner(policy_net, 64)

# shared network
learner_network_id = ray.put(learner)


# actor network, environments initialization
actor_list = [Actor.remote(get_network(learner_network_id), i, 0.5) for i in range(num_actors)]



In [21]:
batch_size = 64
demo_prob = 0.5

(pid=36868) /home/kukjin/anaconda3/envs/minerl/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=36868)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=36871) /home/kukjin/anaconda3/envs/minerl/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=36871)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [ ]:
actor_list[0].make_inveractive.remote()

In [23]:
memory_id = ray.put(memory)

In [24]:
explore = [actor.explore.remote(get_network(learner_network_id), ray.get(memory_id)) for actor in actor_list]

        

(pid=36871) MineRL agent is public, connect on port 12340 with Minecraft 1.11
(pid=36871) Traceback (most recent call last):
(pid=36871)   File "python/ray/_raylet.pyx", line 488, in ray._raylet.execute_task
(pid=36871)   File "python/ray/_raylet.pyx", line 495, in ray._raylet.execute_task
(pid=36871)   File "python/ray/_raylet.pyx", line 505, in ray._raylet.execute_task
(pid=36871)   File "python/ray/_raylet.pyx", line 449, in ray._raylet.execute_task.function_executor
(pid=36871)   File "/home/kukjin/anaconda3/envs/minerl/lib/python3.7/site-packages/ray/_private/function_manager.py", line 556, in actor_method_executor
(pid=36871)     return method(__ray_actor, *args, **kwargs)
(pid=36871)   File "<ipython-input-10-d1758773fc4d>", line 93, in explore
(pid=36871)   File "<ipython-input-10-d1758773fc4d>", line 129, in append_sample
(pid=36871)   File "<ipython-input-6-36415015b796>", line 17, in add
(pid=36871)   File "/home/kukjin/Study/RL/소프트웨어 융합 캡스톤 디자인/ApexDQFD/st.py", line 60, in 

(pid=36871) 0
(pid=36871) 0
(pid=36871) 0
(pid=36871) 0
(pid=36871) 0
(pid=36871) 0
(pid=36871) 0
(pid=36871) 0
(pid=36871) 0


(pid=36868) MineRL agent is public, connect on port 12341 with Minecraft 1.11
(pid=36868) Traceback (most recent call last):
(pid=36868)   File "python/ray/_raylet.pyx", line 488, in ray._raylet.execute_task
(pid=36868)   File "python/ray/_raylet.pyx", line 495, in ray._raylet.execute_task
(pid=36868)   File "python/ray/_raylet.pyx", line 505, in ray._raylet.execute_task
(pid=36868)   File "python/ray/_raylet.pyx", line 449, in ray._raylet.execute_task.function_executor
(pid=36868)   File "/home/kukjin/anaconda3/envs/minerl/lib/python3.7/site-packages/ray/_private/function_manager.py", line 556, in actor_method_executor
(pid=36868)     return method(__ray_actor, *args, **kwargs)
(pid=36868)   File "<ipython-input-10-d1758773fc4d>", line 93, in explore
(pid=36868)   File "<ipython-input-10-d1758773fc4d>", line 129, in append_sample
(pid=36868)   File "<ipython-input-6-36415015b796>", line 17, in add
(pid=36868)   File "/home/kukjin/Study/RL/소프트웨어 융합 캡스톤 디자인/ApexDQFD/st.py", line 60, in 

(pid=36868) 0
(pid=36868) 0
(pid=36868) 0
(pid=36868) 0
(pid=36868) 0
(pid=36868) 0
(pid=36868) 0
(pid=36868) 0
(pid=36868) 0


In [ ]:
    for i in range(1000):
        if(ray.get(memory.size.remote()) > 1000):
            learner.update_network(memory, demos, batch_size, demo_prob, optimizer)
            print("learner network updated")

        if (i % 5 == 0 and i != 0):
            learner.update_target_networks()
            print("learner target network updated")

In [ ]:
ray.shutdown()

In [ ]:
env_list = []

def make_envs(number):
    for i in range(number):
        env = gym.make("MineRLTreechop-v0")
        port_number = int("12340")+i
        env.make_interactive(port=port_number, realtime=False)
        env_list.append(env)

make_envs(3)

In [ ]:
ray.util.register_serializer(env_list)

In [ ]:
@ray.remote
def action(env):
    env.reset()
    for i in range(1000):
        action = env.action_space.sample()
        env.step(action)
    

In [ ]:
env_id_list = [ray.put(env) for env in env_list]

In [ ]:
action_list = [action.remote(env) for env in env_list]

In [ ]:
env_list[0].reset()

In [ ]:
for i in range(10000):
    action = env_list[0].action_space.sample()    
    env_list[0].step(action)
